In [9]:
# 比赛链接 https://www.kaggle.com/c/facebook-v-predicting-check-ins
# 需要点击右上角参加比赛按钮后，才可以下载 数据

In [10]:
import pandas as pd

# 读取FBlocation文件夹下的train.csv
data = pd.read_csv("FBlocation/train.csv")

print(data.shape)
# 预处理：去除有缺失值的行
data = data.dropna()

# 查看数据基本信息
print("删除缺失值后数据形状：", data.shape)
print("前5行预览：")
print(data.head())



(29118021, 6)
删除缺失值后数据形状： (29118021, 6)
前5行预览：
   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949


In [11]:
# 1、缩小数据,查询数据,为了减少计算时间
data = data.query("x > 1.0 &  x < 2.25 & y > 2.5 & y < 3.75")
print(data.shape)

(450045, 6)


In [12]:
# 2、处理数据特征和标签
# 提取标签列
y = data['place_id']

# 删除不需要的列，保留特征列
X = data.drop(['row_id', 'place_id'], axis=1)

# 处理time列，转换为datetime格式并提取时间特征
X['time'] = pd.to_datetime(X['time'], unit='s')
X['day'] = X['time'].dt.day  # 提取日期中的天数
X['hour'] = X['time'].dt.hour  # 提取时间中的小时
X['weekday'] = X['time'].dt.weekday  # 提取星期几（0=周一，6=周日）

print(X)
print('-'*50)
# 删除原始time列，保留新的时间特征
X = X.drop(['time'], axis=1)

print("特征矩阵形状：", X.shape,type(X))
print("标签向量形状：", y.shape,type(y))
print("特征列：", X.columns.tolist())
print("前5行特征数据：")
print(X.head())


               x       y  accuracy                time  day  hour  weekday
31        1.5702  2.8070        47 1970-01-05 03:42:08    5     3        0
74        1.8616  2.8019       133 1970-01-09 21:36:42    9    21        4
242       1.8489  3.6059       844 1970-01-09 23:45:27    9    23        4
376       2.1340  3.1104        52 1970-01-09 00:25:27    9     0        4
390       1.9618  2.6769       989 1970-01-08 05:19:45    8     5        3
...          ...     ...       ...                 ...  ...   ...      ...
29117790  1.6311  2.7197        67 1970-01-09 09:00:40    9     9        4
29117831  1.1036  3.0663        28 1970-01-04 11:41:45    4    11        6
29117859  1.4210  3.2826        23 1970-01-05 08:49:24    5     8        0
29117886  1.6139  3.3473       162 1970-01-08 00:30:52    8     0        3
29117989  1.8143  3.7453        70 1970-01-06 00:53:04    6     0        1

[450045 rows x 7 columns]
--------------------------------------------------
特征矩阵形状： (450045, 6) <c

In [13]:
# 3、统计标签数据
# 统计去重后的数目
unique_places = y.nunique()
print(f"去重后的place_id数目：{unique_places}")

# 统计每个place_id的重复数目
place_counts = y.value_counts()
print(f"\n各place_id的重复数目：")
print(place_counts,type(place_counts))
print('-'*50,place_counts[1097200869])
# 显示重复数目的统计信息
print(f"\n重复数目统计：")
print(f"最多重复次数：{place_counts.max()}")
print(f"最少重复次数：{place_counts.min()}")
print(f"平均重复次数：{place_counts.mean():.2f}")


去重后的place_id数目：8213

各place_id的重复数目：
place_id
3659348746    1609
5351837004    1605
1472576109    1375
9109558344    1354
4588024179    1348
              ... 
7430409324       1
8724544550       1
4102559369       1
7852858307       1
1623323150       1
Name: count, Length: 8213, dtype: int64 <class 'pandas.core.series.Series'>
-------------------------------------------------- 1121

重复数目统计：
最多重复次数：1609
最少重复次数：1
平均重复次数：54.80


In [14]:
# 4、去除只去了一次的place_id的样本
# 找出重复次数大于2的place_id
places_to_keep = place_counts[place_counts > 100].index

# 过滤数据，只保留重复次数大于2的place_id对应的样本
mask = y.isin(places_to_keep)
X_filtered = X[mask]
y_filtered = y[mask]

print(f"过滤前样本数量：{len(X)}")
print(f"过滤后样本数量：{len(X_filtered)}")
print(f"过滤前place_id数量：{y.nunique()}")
print(f"过滤后place_id数量：{y_filtered.nunique()}")
print(len(y))

# 更新X和y
X = X_filtered
y = y_filtered

print(f"\n过滤后的数据形状：")
print(f"特征矩阵形状：{X.shape}")
print(f"标签向量形状：{y.shape}")


过滤前样本数量：450045
过滤后样本数量：368258
过滤前place_id数量：8213
过滤后place_id数量：1170
450045

过滤后的数据形状：
特征矩阵形状：(368258, 6)
标签向量形状：(368258,)


In [15]:
# 5、数据集划分和KNN训练评估
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# 划分训练集和测试集，测试集占15%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

print(f"训练集样本数量：{len(X_train)}")
print(f"测试集样本数量：{len(X_test)}")
print(f"训练集place_id数量：{y_train.nunique()}")
print(f"测试集place_id数量：{y_test.nunique()}")

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  #fit_transform会在训练集计算均值和方差
X_test_scaled = scaler.transform(X_test)   #不会再次计算均值和方差，而是直接进行标准化

# 创建KNN分类器
knn = KNeighborsClassifier(n_neighbors=5,weights='distance')

# 训练模型
print("\n开始训练KNN模型...")
knn.fit(X_train_scaled, y_train)

# 计算准确率
accuracy = knn.score(X_test_scaled, y_test)
print(f"\nKNN模型准确率：{accuracy:.4f}")


训练集样本数量：313019
测试集样本数量：55239
训练集place_id数量：1170
测试集place_id数量：1170

开始训练KNN模型...

KNN模型准确率：0.4035


#  网格搜索

In [16]:
# 使用网格搜索找到最佳的n_neighbors参数
from sklearn.model_selection import GridSearchCV

# 定义参数网格
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'] #uniform: 每个邻居的权重相同 distance: 根据距离进行加权，距离越近权重越大，距离越远权重越小
}

# 创建KNN分类器
knn_grid = KNeighborsClassifier()

# 创建网格搜索对象，使用5折交叉验证
grid_search = GridSearchCV(
    estimator=knn_grid,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# 执行网格搜索
print("开始网格搜索...")
grid_search.fit(X_train_scaled, y_train)

# 输出最佳参数和最佳得分
print(f"\n最佳参数：{grid_search.best_params_}")
print(f"最佳交叉验证得分：{grid_search.best_score_:.4f}")

# 使用最佳参数在测试集上评估
best_knn = grid_search.best_estimator_
test_accuracy = best_knn.score(X_test_scaled, y_test)
print(f"最佳模型在测试集上的准确率：{test_accuracy:.4f}")

# 显示所有参数组合的结果
print("\n所有参数组合的结果：")
results = grid_search.cv_results_
print(results)
for i in range(len(results['params'])):
    print(f"n_neighbors={results['params'][i]['n_neighbors']}: "
          f"平均得分={results['mean_test_score'][i]:.4f} "
          f"(±{results['std_test_score'][i]:.4f})")


开始网格搜索...
Fitting 3 folds for each of 10 candidates, totalling 30 fits

最佳参数：{'n_neighbors': 5, 'weights': 'distance'}
最佳交叉验证得分：0.3706
最佳模型在测试集上的准确率：0.4035

所有参数组合的结果：
{'mean_fit_time': array([1.01096813, 0.96602257, 1.14323036, 0.99622655, 1.16013002,
       1.79481276, 1.7278951 , 1.28030809, 1.44165595, 0.96884267]), 'std_fit_time': array([0.23915837, 0.26418102, 0.29678957, 0.27259934, 0.36936753,
       0.80337528, 0.56012485, 0.6742413 , 0.6201853 , 0.25823412]), 'mean_score_time': array([10.95676589, 17.21314311, 10.18730688, 21.94262552, 13.60832079,
       25.55475569, 12.87464078, 23.64131991, 13.21195078, 18.82611982]), 'std_score_time': array([2.06123852, 2.5822819 , 0.78262118, 4.17015581, 2.00847864,
       2.0519112 , 2.0990766 , 2.14573538, 0.35489029, 1.59394142]), 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 7, 7, 9, 9, 11, 11],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value=999999)